# getting data onto GEMS grid

this notebook exists to document the exploration of methods for getting data onto the GEMS grid. Focus here is predominanatly on getting raster data to the GEMS grid. Following Jensen (2005) there are two basic models for converting rasters:
  1. Input-to-Out Model: In this model, the transformation are done in the forward direction. All coordinates in the source cooridnate system are transformed to the target system, and then the target raster is created using those points and data. This can be thought of as a 'push' or 'put' type model. The challenge with this model is how to deal with the fact that once coordinates are converted to target, they may not align  with target cells (e.g. raster index 5, 2 maps to a cell boundary in the output)
  2. Output-to-input model: In this model, the transformations are done in the reverse direction. Dimensions of the output are created, and essentially the mapping is backwards. Coordinates in the output raster are used to identify cooresponding coordinates in the input raster. Often combined with 'nearest neighbor' operations. This can be thought of as a 'pull' or 'get' type of model. The disadvanted here has generally been that that output grid must be known or created in advance. 
  
Many of the tools (e.g. GDAL, ESRI projection toolkit) generally operate in the forward model. Input images and metadata are used with analyst input to create the target raster, and data are pushed through the process in the forward direction. 

Options like ESRI's 'snap to raster' give the ability to ensure that output grids are all consistent, thereby enabling stacking of grids etc. Mechanism there is unclear, but results are akin to the `output-to-input` model above. 

In this notebook, the aim is to investigate standard open source tools (gdal mainly) to see how we might generate output rasters using a process that is more similar to the `output-to-input` model. A challenge with these tools is that grids are almost always tansformed using an affine model, from the upper left corner of the input, with output image dimensiond derived from analyst input. The result is often an output raster that  imperfectly aligns with the GEMS grid.

Consider the current process that generated the GEMS grid version of the Sacks crop calendar. The input is global in extent, and cutting the inputs down to match the GEMS grid extent introduces challenges. Either the image dimensions don't match (e.g. are more rows/columns in output raster than allowable) or the corners or centroids don't align perfectly. For example, the current Sacks tifs have these properties:

      Corner Coordinates
      Upper Left  (-17367530.445, 7314540.831) (180d 0' 0.00"W, 85d 2'40.44"N)
      Lower Left  (-17367530.445,-7314540.830) (180d 0' 0.00"W, 85d 2'40.44"S)
      Upper Right (17367530.445, 7314540.831) (180d 0' 0.00"E, 85d 2'40.44"N)
      Lower Right (17367530.445,-7314540.830) (180d 0' 0.00"E, 85d 2'40.44"S)
      Center      (  -0.0002814,   0.0000334) (  0d 0' 0.00"W,  0d 0' 0.00"N)

While the corners align nicely, note that the center of the image is slightly different from 0, certainly its within rounding, but this has the potential to cause data alignment issues.

`gdalwarp` has several options that might be useful here, allowing us to avoid writing our own `output-to-input` algorythm.



`-te` : target extent. This is useful for getting the target bounding box extents. Used alone, it can result in misaligned grid centers. For example, using gdal with `-te` alone on the Sacks data results in these properties:

      Corner Coordinates
      Upper Left  (-17367530.445, 7314540.831) (180d 0' 0.00"W, 85d 2'40.44"N)
      Lower Left  (-17367530.445,-7314540.830) (180d 0' 0.00"W, 85d 2'40.44"S)
      Upper Right (17367530.445, 7314540.831) (180d 0' 0.00"E, 85d 2'40.44"N)
      Lower Right (17367530.445,-7314540.830) (180d 0' 0.00"E, 85d 2'40.44"S)
      Center      (  -0.0002814,   0.0000334) (  0d 0' 0.00"W,  0d 0' 0.00"N)
      
Note the process used to arrive at these grids are not quite identical, they generate the same coords.



### Note:
The usual way `te` expects data are:
`te minx miny maxx maxy`

Below, this is not followed, but notice the `tr` (target resolution) argument has a `-` for the `yres` component. This allows for specifiying the options as they might come though in a NetCDF, where the coords are specified in the inverse direction (data often come sorted lowest to highest, but the convention for mapping is North is up, which are positive, and often the vectors are ordered highest to lowest)

#### example command with `te` is:
```
gdalwarp -s_srs EPSG:4326 -t_srs EPSG:6933 -te -17367530.445161372 7314540.830638504 17367530.445161372 -7314540.830638504 -tr 9008.05521 -9008.05521 -r near -of GTiff -overwrite NETCDF:"Maize.crop.calendar.nc":harvest maize_warp_tap_2022_02_15.tif
```

`-tap` : target aligned pixels. This can be used to generate grids which have perfect alignment at the center. Per the documentation, this ensures that coordinate extents are included in the grid. When testing with Sacks calendar, this seemed to have the effect of adding additional two addional columns to the raster.

      Corner Coordinates:
      Upper Left  (-17376538.500,-7314540.831) (179d54'23.90"E, 85d 2'40.44"S)
      Lower Left  (-17376538.500, 7314540.831) (179d54'23.90"E, 85d 2'40.44"N)
      Upper Right (17376538.500,-7314540.831) (179d54'23.90"W, 85d 2'40.44"S)
      Lower Right (17376538.500, 7314540.831) (179d54'23.90"W, 85d 2'40.44"N)
      Center      (   0.0000000,   0.0000000) (  0d 0' 0.01"E,  0d 0' 0.01"N)

#### example command with `te` and `tap`
```
gdalwarp -s_srs EPSG:4326 -t_srs EPSG:6933 -te -17367530.445161372 7314540.830638504 17367530.445161372 -7314540.830638504 -tr 9008.05521 -9008.05521 -r near -of GTiff -tap --overwrite NETCDF:"Maize.crop.calendar.nc":harvest maize_warp_tap_2022_02_15.tif
```

### removing excess columns 

The `tap` option and expectations are not really well described. In the case above when used in conjuntion with `te` is seems to expand the row count by two pixels (left/right). The problem is, the then causes the grid extents to wrap around:

      Corner Coordinates:
      Upper Left  (-17376538.500, 7323548.886) (179d54'23.90"E, 85d55'48.45"N)
      Lower Left  (-17376538.500,-7323548.886) (179d54'23.90"E, 85d55'48.45"S)
      Upper Right (17376538.500, 7323548.886) (179d54'23.90"W, 85d55'48.45"N)
      Lower Right (17376538.500,-7323548.886) (179d54'23.90"W, 85d55'48.45"S)
      Center      (   0.0000000,   0.0000000) (  0d 0' 0.01"E,  0d 0' 0.01"N)
      
Note that rather than having a left boundaries that corrspond to ~ -180.0 degrees, we've entered positive territory again. Effectively, we've wrapped around the edge of the edge of the flat map. 

In [5]:
from gems_grid_lib import grid_dict, level_dict
grid_dict.keys(), level_dict.keys()


(dict_keys(['min_x', 'min_y', 'max_x', 'max_y', 'min_lon', 'min_lat', 'max_lon', 'max_lat']),
 dict_keys(['0', '1', '2', '3', '4', '5', '6']))

### find extent for equivalent of removing 1 pixel each side

In [7]:
grid_dict['min_x'] + level_dict['1']['x_length']



-17358522.38995137

#### example code is:
```
gdalwarp -s_srs EPSG:4326 -t_srs EPSG:6933 -te -17358522.38995137 7314540.830638504 17358522.38995137 -7314540.830638504 -tr 9008.05521 -9008.05521 -r near -of GTiff -tap -overwrite NETCDF:"Maize.crop.calendar.nc":harvest maize_warp_tap_te_adjust_2022_02_16.tif
```

This results in raster with following properties:

      Data axis to CRS axis mapping: 1,2
      Origin = (-17367530.444880001246929,7314540.830520000308752)
      Pixel Size = (9008.055210000000443,-9008.055210000000443)
      
      ...
      
      Corner Coordinates:
      Upper Left  (-17367530.445, 7323548.886) (180d 0' 0.00"W, 85d55'48.45"N)
      Lower Left  (-17367530.445,-7323548.886) (180d 0' 0.00"W, 85d55'48.45"S)
      Upper Right (17367530.445, 7323548.886) (180d 0' 0.00"E, 85d55'48.45"N)
      Lower Right (17367530.445,-7323548.886) (180d 0' 0.00"E, 85d55'48.45"S)
      Center      (   0.0000000,   0.0000000) (  0d 0' 0.01"E,  0d 0' 0.01"N)


In [11]:
print('origin from GEMS grid dictionary: \t{}, {}'.format(grid_dict['min_x'], grid_dict['max_y']))

origin from GEMS grid dictionary: 	-17367530.445161372, 7314540.830638504


#### now I need to get rid of the top/bottom rows

that is grid dimensions don't match. and only get (rounded) agreement down to 3rd sig fig

In [10]:
grid_dict['min_y'] + level_dict['1']['y_length']

-7305532.775428504

### results now match somewhat better
      Size is 3856, 1624
      ...
      
      Origin = (-17367530.444880001246929,7314540.830520000308752)
      Pixel Size = (9008.055210000000443,-9008.055210000000443)
      ...
      
      Corner Coordinates:
      Upper Left  (-17367530.445, 7314540.831) (180d 0' 0.00"W, 85d 2'40.44"N)
      Lower Left  (-17367530.445,-7314540.831) (180d 0' 0.00"W, 85d 2'40.44"S)
      Upper Right (17367530.445, 7314540.831) (180d 0' 0.00"E, 85d 2'40.44"N)
      Lower Right (17367530.445,-7314540.831) (180d 0' 0.00"E, 85d 2'40.44"S)
      Center      (   0.0000000,   0.0000000) (  0d 0' 0.01"E,  0d 0' 0.01"N)

In [12]:
print('origin from GEMS grid dictionary: \t{}, {}'.format(grid_dict['min_x'], grid_dict['max_y']))

origin from GEMS grid dictionary: 	-17367530.445161372, 7314540.830638504


## results seem reasonable

these are reasonable, and do suggest a potential way forward. algorithm could look like:
 + find geographic extents
 + convert to EASE v2
 + identify the extent at GEMS grid level
 + shrink that grid by one row/column on all sides
 + warp using `te` with smaller extent
 + aligment of other grids accomplished with `tap`
 
questions to investigate:
  + above is a symetric example, centered at 0,0. What does the MN case (e.g. David's water quaility data) look like in terms of alignment?
  + are there implications when coordinate data are rounded? if so, how to deal with it? 
  + how to deal with the mosaic issue? david's process is one of mosaic then grid.
  
So the `gdalwarp -te -tap` process seems to emulate a pull, but a true pull, output-to-input algorithm might alleviate some of this all together? not clear how a mosaic is handled there


## true output-to-input

using the `-to` option, can set `bDstToSrc=True` and this will actually do the transformation using an output-to-input algorithm

big question here: the symetrical answer is easy to test. how to test other responses: use `pyproj.Transform` to do the math, then figure out what the responses look like via `gdalwarp`?
  + can certainly generate a couple examples to test. what is a reasonable proceess there?
     + pick a random row, column. take a fixed buffer from there. should be able to calcualte the raster extent. convert to geos, then work through the code to project? 
     + pick a semi random number of sides, with a 'diamter', then do similar to above? 

  + pick some real shapes, solve independelty, then those as the test solutions?